# Import

In [ ]:
import math
import random
import re
import sys
import time
import pickle

import matplotlib.pyplot as plt

import torch
import torch.utils.data

## Dataset

In [ ]:
VRConfig = {
    'rounds': 10,
    'displayInterval': 4000,
    
    'weight_decay': 0.01,
    'honestSize': 50,
    'byzantineSize': 20,
    
    'SEED': 200,
    'fixSeed': True,
}

ijcnn1 dataset

In [ ]:
# dataSetConfig = {
#     'name': 'ijcnn1',
#     'dataSet' : 'ijcnn1',
#     'dataSetSize': 49990,
#     'maxFeature': 22,
#     'findingType': '1',
# }

# VRConfig['SET_SIZE'] = dataSetConfig['dataSetSize']

# SGDConfig = VRConfig.copy()
# SGDConfig['gamma'] = 2e-2

# batchConfig = VRConfig.copy()
# batchConfig['batchSize'] = 50
# batchConfig['gamma'] = 1e-2

# SVRGConfig = VRConfig.copy()
# SVRGConfig['snapshotInterval'] = dataSetConfig['dataSetSize']
# SVRGConfig['gamma'] = 2e-2

# SAGAConfig = VRConfig.copy()
# SAGAConfig['gamma'] = 2e-2

# SARAHConfig = VRConfig.copy()
# SARAHConfig['gamma'] = 2e-2

# ByrD2SAGAConfig = VRConfig.copy()
# ByrD2SAGAConfig['gamma'] = 2e-2

covtype dataset

In [ ]:
dataSetConfig = {
    'name': 'covtype',
    'dataSet' : 'covtype.libsvm.binary.scale',
    'dataSetSize': 581012,
    'maxFeature': 54,
    'findingType': '1',
}

VRConfig['SET_SIZE'] = dataSetConfig['dataSetSize']

SGDConfig = VRConfig.copy()
SGDConfig['gamma'] = 1e-2

batchConfig = VRConfig.copy()
batchConfig['batchSize'] = 50
batchConfig['gamma'] = 5e-3

SVRGConfig = VRConfig.copy()
SVRGConfig['snapshotInterval'] = dataSetConfig['dataSetSize']
SVRGConfig['gamma'] = 1e-2

SAGAConfig = VRConfig.copy()
SAGAConfig['gamma'] = 5e-3

SARAHConfig = VRConfig.copy()
SARAHConfig['gamma'] = 1e-2

ByrD2SAGAConfig = VRConfig.copy()
ByrD2SAGAConfig['gamma'] = 1e-2

In [ ]:
SET_SIZE = dataSetConfig['dataSetSize']
maxFeature = dataSetConfig['maxFeature']
findingType = dataSetConfig['findingType']

CACHE_DIR = './cache/' + dataSetConfig['name'] + '_'
# ====================================================
# 报告函数
def log(*k, **kw):
    timeStamp = time.strftime('[%y-%m-%d %H:%M:%S] ', time.localtime())
    print(timeStamp, end='')
    print(*k, **kw)
    sys.stdout.flush()
    
def logAxis(path, Fmin):
#     return [math.log10(p-Fmin) for p in path]
    return [p-Fmin for p in path]

## Parameter

In [ ]:
# L = np.sum([(scipy.sparse.linalg.norm(X[i, :]) + 1)
#             ** 2 for i in range(X.shape[0])])
# L = Lambda + 1/(4*SET_SIZE) * L

torch.manual_seed(VRConfig['SEED'])#为CPU设置随机种子

w0 = torch.zeros(maxFeature + 1, dtype=torch.float64)
w0 = torch.nn.init.normal_(w0)

## Load dataset

In [ ]:
class SVM_dataSet(torch.utils.data.Dataset):
    def __init__(self, **dataSetConfig):
        super(SVM_dataSet, self).__init__()
        log('开始加载数据集')
        self.X = torch.zeros((SET_SIZE, maxFeature), dtype=torch.float64)
        self.Y = torch.zeros((SET_SIZE), dtype=torch.float64)
        __dir__ = '.'
        dataFile = __dir__ + '/dataset/' + dataSetConfig['dataSet']

        with open(dataFile, 'r') as f:
            posCount = 0
            negCount = 1
            for (line, vector) in enumerate(f):
                (cat, data) = vector.split(' ', 1)
                if cat == findingType:
                    self.Y[line] = 1
                    posCount += 1
                else:
                    self.Y[line] = 0
                    negCount += 1
                for piece in data.strip().split(' '):
                    match = re.search(r'(\S+):(\S+)', piece)
                    feature = int(match.group(1)) - 1  # 数据集从1开始
                    value = float(match.group(2))
                    # 插入矩阵
                    self.X[line][feature] = value
        log('加载数据集完成({})，正类：{}个，负类：{}个'.format(dataSetConfig['dataSet'], posCount, negCount))
        
        # 设置随机取样
        self.__RR = False
        self.__order = list(range(SET_SIZE))
    def randomReshuffle(self):
        self.__RR = True
        random.shuffle(self.__order)
    def resetShuffle(self):
        self.RR = False
    def __getitem__(self, index):
        if self.__RR:
            i = self.__order[index]
            return self.X[i], self.Y[i]
        else:
            return self.X[index], self.Y[index]
    def __len__(self):
        return SET_SIZE

In [ ]:
dataset = SVM_dataSet(**dataSetConfig)

In [ ]:
# L = np.sum([(x.norm().item() + 1)** 2 for x, _ in dataset])
# L = VRConfig['weight_decay'] + 1/(4*SET_SIZE) * L

## Loss function

In [ ]:
def accuracy(w, dataset):
    correct = 0
    for data, label in dataset:
        pre = LogisticRegression(w, data) > 0.5
        correct += (pre.type(torch.uint8) == label.type(torch.uint8)).item()
    return correct / len(dataset)
def F(w, dataset, weight_decay):
    loss = 0
    for data, label in dataset:
        predict = LogisticRegression(w, data)
        loss += torch.nn.functional.binary_cross_entropy(predict, label)
    loss /= len(dataset)
    loss += weight_decay * torch.norm(w)**2 / 2
    return loss.item()
def G(w, dataset, weight_decay):
    G = torch.zeros_like(w, requires_grad=False, dtype=torch.float64)
    g = torch.zeros_like(w, requires_grad=False, dtype=torch.float64)
    for index in range(len(dataset)):
        x, y = dataset[index]
        predict = LogisticRegression(w, x)

        err = -(y-predict).data
        g[:-1] = err*x
        g[-1] = err
        G.add_(1/len(dataset), g)
    G.add_(weight_decay, w)
    return G
def LogisticRegression(w, x):
    out = w[:-1].dot(x) + w[-1]
    return torch.sigmoid(out)

In [ ]:
def getVarience(w_local, honestSize):
    avg = w_local[:honestSize].mean(dim=0)
    s = 0
    for w in w_local[:honestSize]:
        s += (w - avg).norm()**2
    s /= honestSize
    return s.item()

In [ ]:
def getOuterVariation(w_min, honestSize):
    # 数据分片
    pieces = [(i*len(dataset)) // honestSize for i in range(honestSize+1)]
    dataPerNode = [pieces[i+1] - pieces[i] for i in range(honestSize)]
    
    gradients = []
    for node in range(honestSize):
        gradient = torch.zeros_like(w_min)
        for index in range(pieces[node], pieces[node+1]):
            x, y = dataset[index]
            # 更新梯度表
            predict = LogisticRegression(w_min, x)

            err = (predict-y).data
            gradient[:-1].add_(err*x)
            gradient[-1].add_(err)
        gradient.div_(dataPerNode[node])
        gradients.append(gradient)
    gradients = torch.stack(gradients)
    outerVariation = getVarience(gradients, honestSize)
    return outerVariation

# Aggregation methods

In [ ]:
def mean(wList):
    return torch.mean(wList, dim=0)

In [ ]:
def gm(wList):
    max_iter = 80
    tol = 1e-5
    guess = torch.mean(wList, dim=0)
    for _ in range(max_iter):
        dist_li = torch.norm(wList-guess, dim=1)
        for i in range(len(dist_li)):
            if dist_li[i] == 0:
                dist_li[i] = 1
        temp1 = torch.sum(torch.stack([w/d for w, d in zip(wList, dist_li)]), dim=0)
        temp2 = torch.sum(1/dist_li)
        guess_next = temp1 / temp2
        guess_movement = torch.norm(guess - guess_next)
        guess = guess_next
        if guess_movement <= tol:
            break
    return guess

In [ ]:
def gm7(wList):
    max_iter = 80
    tol = 1e-7
    guess = torch.mean(wList, dim=0)
    for _ in range(max_iter):
        dist_li = torch.norm(wList-guess, dim=1)
        for i in range(len(dist_li)):
            if dist_li[i] == 0:
                dist_li[i] = 1
        temp1 = torch.sum(torch.stack([w/d for w, d in zip(wList, dist_li)]), dim=0)
        temp2 = torch.sum(1/dist_li)
        guess_next = temp1 / temp2
        guess_movement = torch.norm(guess - guess_next)
        guess = guess_next
        if guess_movement <= tol:
            break
    return guess

In [ ]:
def Krum_(nodeSize, byzantineSize):
    honestSize = nodeSize - byzantineSize
    dist = torch.zeros(nodeSize, nodeSize, dtype=torch.float32)
    def Krum(wList):
        for i in range(nodeSize):
            for j in range(i, nodeSize):
                distance = wList[i].data - wList[j].data
                distance = (distance*distance).sum()
                distance = -distance # 两处都是取距离的最小值，需要改成负数
                dist[i][j] = distance.data
                dist[j][i] = distance.data
        k = nodeSize - byzantineSize - 2 + 1 # 算上自己和自己的0.00
        topv, _ = dist.topk(k=k, dim=1)
        sumdist = topv.sum(dim=1)
        resindex = sumdist.topk(1)[1].squeeze()
        return wList[resindex]
    return Krum

In [ ]:
def median(wList):
    return wList.median(dim=0)[0]

# Compression functions

In [ ]:
def Quantized_l1sign(w):
    # w: shape(NodeSize, parameter length)
    
    nodesize, paralength = w.size()
    w_l1 = torch.norm(w, p=1, dim=1).view(nodesize, 1)
    w_l1 = w_l1 / paralength
    w = torch.sign(w) * w_l1
    
    return w

In [ ]:
def Quantized_topk(w):
    # w: shape(NodeSize, parameter length)
    ratio = 0.1
    
    nodesize, paralength = w.size()
    k = round(paralength * ratio)
    _, index = torch.topk(abs(w), k, dim=1) #obtain the locations of topk absolute values
    w_topk = torch.gather(w, 1, index) # extract the topk elements from w
    #w_topk = paralength / k * w_topk
    # obtain topk matrix of w with other elements being zero
    w = torch.zeros(nodesize, paralength, dtype=torch.float64).scatter_(1, index, w_topk) 
    
    #w = w/ratio
    
    
    return w

In [ ]:
def Quantized_randk(w):
    # w: shape(NodeSize, parameter length)
    ratio = 0.1
    
    nodesize, paralength = w.size()
    k = round(paralength * ratio)
    index = torch.zeros(nodesize, paralength, dtype=torch.int64)
    for i in range(0,nodesize):
        index[i,:] = torch.randperm(paralength)
    index = index[:,0:k]
    w_randk = torch.gather(w, 1, index) # extract the randk elements from w
    # obtain randk matrix of w with other elements being zero
    w = torch.zeros(nodesize, paralength, dtype=torch.float64).scatter_(1, index, w_randk) 
    
    w = w/ratio
    
    return w

In [ ]:
def norm_threshold(w):
    # w: shape(NodeSize, parameter length)
    
    nodesize, paralength = w.size()
    beta = int(nodesize * 0.3)
    w_l1 = torch.norm(w, p=1, dim=1)
    
    _, index = torch.sort(w_l1)
    index_sele = index[:(nodesize - beta)]
    w = torch.index_select(w, 0, index_sele)
    
    return w

# Optimization methods

## Central SAGA

In [ ]:
def CentralSAGA(w0, gamma, weight_decay, rounds=10, displayInterval=1000, SEED=100, fixSeed=False, **kw):

    # 初始化
    w = w0.clone().detach()
    
    store = torch.zeros([SET_SIZE, w.size(0)], requires_grad=False, dtype=torch.float64)
    for index in range(SET_SIZE):
        x, y = dataset[index]
        predict = LogisticRegression(w, x)

        err = (predict-y).data
        store[index][:-1] = err*x
        store[index][-1] = err
        store[index].add_(weight_decay, w)

    G_avg = torch.mean(store, dim=0)
    path = [F(w, dataset, weight_decay)]
    
    # 中间变量分配空间
    new_G = torch.zeros(w0.size(), dtype=torch.float64)
    
    log('[SAGA]初始 loss={:.6f}, accuracy={:.2f} gamma={:}'.format(path[0], accuracy(w, dataset), gamma))
    log('开始迭代')
    for r in range(rounds):
        for k in range(displayInterval):
            # 更新梯度表
            index = random.randint(0, SET_SIZE-1)

            x, y = dataset[index]
            predict = LogisticRegression(w, x)
            
            # 计算梯度
            old_G = store[index]
            err = (predict-y).data
            new_G[:-1] = err*x
            new_G[-1] = err
            new_G.add_(weight_decay, w)
            
            gradient = new_G.data - old_G.data + G_avg.data
            
            G_avg.add_(1 / SET_SIZE, new_G.data - old_G.data)
            store[index] = new_G.data
            w.data.add_(-gamma, gradient.data)
            
        loss = F(w, dataset, weight_decay)
        acc = accuracy(w, dataset)
        path.append(loss)
        log('[SAGA]已迭代 {}/{} rounds (interval: {:.0f}), loss={:.9f}, accuracy={:.2f}'.format(
            r+1, rounds, displayInterval, loss, acc
        ))
    return w, path, []

In [ ]:
def SAGA_min(w0, dataset, gamma, weight_decay, epoch=1, **kw):

    # 初始化
    w = w0.clone().detach()
    
    store = torch.zeros([SET_SIZE, w.size(0)], requires_grad=False, dtype=torch.float64)
    for index in range(SET_SIZE):
        x, y = dataset[index]
        predict = LogisticRegression(w, x)

        err = -(y-predict).data
        store[index][:-1] = err*x
        store[index][-1] = err
        store[index].add_(weight_decay, w)

    G_avg = torch.mean(store, dim=0)
    
    # 中间变量分配空间
    new_G = torch.zeros(w0.size(), dtype=torch.float64)
    for e in range(epoch):
        for _ in range(SET_SIZE):
            # 更新梯度表
            index = random.randint(0, SET_SIZE-1)

            x, y = dataset[index]
            predict = LogisticRegression(w, x)
            
            # 计算梯度
            old_G = store[index]
            err = -(y-predict).data
            new_G[:-1] = err*x
            new_G[-1] = err
            new_G.add_(weight_decay, w)
            
            gradient = new_G.data - old_G.data + G_avg.data
            
            G_avg.add_(1 / SET_SIZE, new_G.data - old_G.data)
            store[index] = new_G.data
            w.data.add_(-gamma, gradient.data)
        log('[SAGA]已迭代{:.0f}/{:.0f}趟'.format(e+1, epoch))
    
    return w

## SGD

In [ ]:
def SGD(w0, gamma, aggregate, weight_decay, honestSize=0, byzantineSize=0, attack=None,
            rounds=10, displayInterval=1000, SEED=100, fixSeed=False, **kw):
    assert byzantineSize == 0 or attack != None
    assert honestSize != 0
    
    if fixSeed:
        random.seed(SEED)

    nodeSize = honestSize + byzantineSize
    
    # 初始化
    w = w0.clone().detach()

    # 数据分片
    pieces = [(i*len(dataset)) // honestSize for i in range(honestSize+1)]
    dataPerNode = [pieces[i+1] - pieces[i] for i in range(honestSize)]

    path = [F(w, dataset, weight_decay)]
    variencePath = []
    log('[SGD]初始 loss={:.6f}, accuracy={:.2f} gamma={:}'.format(path[0], accuracy(w, dataset), gamma))
    
    # 中间变量分配空间
    new_G = torch.zeros_like(w0, dtype=torch.float64)
    message = torch.zeros(nodeSize, len(w0), dtype=torch.float64)
    
    quan = torch.zeros(nodeSize, len(w0), dtype=torch.float64)
    mem = torch.zeros(nodeSize, len(w0), dtype=torch.float64)
    quanc = torch.zeros(1, len(w0), dtype=torch.float64)
    memc = torch.zeros(1, len(w0), dtype=torch.float64)
    
    message1 = torch.zeros(honestSize, len(w0), dtype=torch.float64)
    message2 = torch.zeros(byzantineSize, len(w0), dtype=torch.float64)
    H = torch.zeros(nodeSize, len(w0), dtype=torch.float64)
    H1 = torch.ones(honestSize, len(w0), dtype=torch.float64)
    H2 = torch.ones(byzantineSize, len(w0), dtype=torch.float64)
    quan1 = torch.zeros(honestSize, len(w0), dtype=torch.float64)
    quan2 = torch.zeros(byzantineSize, len(w0), dtype=torch.float64)
    alpha = 1e-1

    log('开始迭代')
    for r in range(rounds):
        for k in range(displayInterval):
            # 诚实节点更新
            for node in range(honestSize):
                index = random.randint(pieces[node], pieces[node+1]-1)

                x, y = dataset[index]
                # 更新梯度表
                predict = LogisticRegression(w, x)
                err = (predict-y).data
                new_G[:-1] = err*x
                new_G[-1] = err
                new_G.add_(weight_decay, w)
                
                gradient = new_G
                
                message[node].copy_(gradient.data)
            
            #Quantize the honest information  
#             quan = Quantized_topk(message + mem) 
#             mem = message + mem - quan
#             message = quan

            # 同步
            # Byzantine攻击
            if attack != None:
                attack(message, byzantineSize)
            
            
              #Quantize all the information, double pass + error feedback
#             quan = Quantized_topk(message + mem) 
#             mem = message + mem - quan
#             message = quan https://live.bilibili.com/blackboard/activity-Sj6iU9MnS.html?visit_id=6x2jf6n3oxkw
#             g = aggregate(message)
#             quanc = Quantized_topk(g + memc)
#             memc = g + memc - quanc
#             g = quanc
#             g = g.squeeze()
            
    
            #Quantize all the information, single pass + error feedback
#             quan = Quantized_l1sign(message + mem) 
#             mem = message + mem - quan
#             message = quan          
#             g = aggregate(message)

             #Quantize all the information, single pass
#             message1 = message[0:honestSize]
#             message2 = message[honestSize:]
#             quan1 = Quantized_randk(message1)
#             quan2 = Quantized_topk(message2)
#             message = torch.cat((quan1, quan2), 0)
#             g = aggregate(message)

            #Gradient Norm Threshold based method
#             message1 = message[0:honestSize]
#             message2 = message[honestSize:]
#             quan1 = Quantized_randk(message1)
#             quan2 = Quantized_topk(message2)
#             message = torch.cat((quan1, quan2), 0)
#             message_select = norm_threshold(message)          
#             g = aggregate(message_select)
            
    
            #SignSGD
            message = torch.sign(message)
            g = aggregate(message)
        
            #DIANA type Quantization, single pass
#             message1 = message[0:honestSize]
#             message2 = message[honestSize:]
#             delta1 = message1 - H1
#             quan1 = Quantized_randk(delta1)
#             message1 = H1 + quan1
#             delta2 = message2 - H2
#             quan2 = Quantized_topk(delta2)
#             message2 = H2 + quan2
#             H1 = H1 + alpha * quan1
#             H2 = H2 + alpha * quan2
#             message = torch.cat((message1, message2), 0)
#             g = aggregate(message)
            
#             delta = message - H
#             quan = Quantized_randk(delta)
#             message = H + quan
#             H = H + alpha * quan
#             g = aggregate(message)
            

#             g = aggregate(message)
            w.add_(-gamma, g.data)
        
        loss = F(w, dataset, weight_decay)
        acc = accuracy(w, dataset)
        path.append(loss)
        var = getVarience(message, honestSize)
        variencePath.append(var)
        log('[SGD]已迭代 {}/{} rounds (interval: {:.0f}), loss={:.9f}, accuracy={:.2f}, var={:.9f}'.format(
            r+1, rounds, displayInterval, loss, acc, var
        ))
    return w, path, variencePath


## BatchSGD

In [ ]:
def BatchSGD(w0, gamma, aggregate, weight_decay, honestSize=0, byzantineSize=0, attack=None, batchSize=50,
            rounds=10, displayInterval=1000, SEED=100, fixSeed=False, **kw):
    assert byzantineSize == 0 or attack != None
    assert honestSize != 0
    
    if fixSeed:
        random.seed(SEED)

    nodeSize = honestSize + byzantineSize
    
    # 初始化
    w = w0.clone().detach()

    # 数据分片
    pieces = [(i*len(dataset)) // honestSize for i in range(honestSize+1)]
    dataPerNode = [pieces[i+1] - pieces[i] for i in range(honestSize)]

    path = [F(w, dataset, weight_decay)]
    variencePath = []
    log('[BatchSGD]初始 loss={:.6f}, accuracy={:.2f} gamma={:}'.format(path[0], accuracy(w, dataset), gamma))
    
    # 中间变量分配空间
    new_G = torch.zeros_like(w0, dtype=torch.float64)
    message = torch.zeros(nodeSize, len(w0), dtype=torch.float64)

    log('开始迭代')
    for r in range(rounds):
        for k in range(displayInterval):
            # 诚实节点更新
            for node in range(honestSize):
                gradient = torch.zeros_like(new_G)
                for b in range(batchSize):
                    index = random.randint(pieces[node], pieces[node+1]-1)

                    x, y = dataset[index]
                    # 更新梯度表
                    predict = LogisticRegression(w, x)
                    err = (predict-y).data
                    new_G[:-1] = err*x
                    new_G[-1] = err
                    new_G.add_(weight_decay, w)
                    gradient.add_(1/batchSize, new_G)
                message[node].copy_(gradient.data)

            # 同步
            # Byzantine攻击
            if attack != None:
                attack(message, byzantineSize)
            g = aggregate(message)
            w.add_(-gamma, g.data)
            
        loss = F(w, dataset, weight_decay)
        acc = accuracy(w, dataset)
        path.append(loss)
        var = getVarience(message, honestSize)
        variencePath.append(var)
        log('[BatchSGD]已迭代 {}/{} rounds (interval: {:.0f}), loss={:.9f}, accuracy={:.2f}, var={:.9f}'.format(
            r+1, rounds, displayInterval, loss, acc, var
        ))
    return w, path, variencePath


## SAGA

In [ ]:
def SAGA(w0, gamma, aggregate, weight_decay, honestSize=0, byzantineSize=0, attack=None, 
            rounds=10, displayInterval=1000, SEED=100, fixSeed=False, **kw):
    assert byzantineSize == 0 or attack != None
    assert honestSize != 0
    
    if fixSeed:
        random.seed(SEED)

    nodeSize = honestSize + byzantineSize
    
    # 初始化
    w = w0.clone().detach()

    store = torch.zeros([len(dataset), w.size(0)], requires_grad=False, dtype=torch.float64)
    for index in range(len(dataset)):
        x, y = dataset[index]
        predict = LogisticRegression(w, x)

        err = (predict-y).data
        store[index][:-1] = err*x
        store[index][-1] = err
        store[index].add_(weight_decay, w)

    # 数据分片
    pieces = [(i*len(dataset)) // honestSize for i in range(honestSize+1)]
    dataPerNode = [pieces[i+1] - pieces[i] for i in range(honestSize)]

    G_avg = torch.stack([
        store[pieces[i]:pieces[i+1]].mean(dim=0) for i in range(honestSize)
    ])
    path = [F(w, dataset, weight_decay)]
    variencePath = []
    log('[SAGA]初始 loss={:.6f}, accuracy={:.2f} gamma={:}'.format(path[0], accuracy(w, dataset), gamma))
    
    # 中间变量分配空间
    new_G = torch.zeros_like(w0, dtype=torch.float64)
    message = torch.zeros(nodeSize, len(w0), dtype=torch.float64)
    
    quan = torch.zeros(nodeSize, len(w0), dtype=torch.float64)
    mem = torch.zeros(nodeSize, len(w0), dtype=torch.float64)
    quanc = torch.zeros(1, len(w0), dtype=torch.float64)
    memc = torch.zeros(1, len(w0), dtype=torch.float64)
    
    message1 = torch.zeros(honestSize, len(w0), dtype=torch.float64)
    message2 = torch.zeros(byzantineSize, len(w0), dtype=torch.float64)
    H = torch.zeros(nodeSize, len(w0), dtype=torch.float64)
    H1 = 0.01*torch.ones(honestSize, len(w0), dtype=torch.float64)
    H2 = 0.01*torch.ones(byzantineSize, len(w0), dtype=torch.float64)
    quan1 = torch.zeros(honestSize, len(w0), dtype=torch.float64)
    quan2 = torch.zeros(byzantineSize, len(w0), dtype=torch.float64)
    alpha = 0.001

    log('开始迭代')
    for r in range(rounds):
        for k in range(displayInterval):
            # 诚实节点更新
            for node in range(honestSize):
                index = random.randint(pieces[node], pieces[node+1]-1)

                x, y = dataset[index]
                # 更新梯度表
                predict = LogisticRegression(w, x)

                old_G = store[index]
                err = (predict-y).data
                new_G[:-1] = err*x
                new_G[-1] = err
                new_G.add_(weight_decay, w)

                gradient = new_G.data - old_G.data + G_avg[node].data

                G_avg[node].add_(1 / dataPerNode[node],
                                 new_G.data - old_G.data)
                store[index] = new_G.data

                message[node].copy_(gradient.data)
              
            #Quantize the honest information  
#             quan = Quantized_topk(message + mem) 
#             mem = message + mem - quan
#             message = quan

            # 同步
            # Byzantine攻击
            if attack != None:
                attack(message, byzantineSize)
     
    
            #Quantize all the information, double pass + error feedback
#             quan = Quantized_topk(message + mem) 
#             mem = message + mem - quan
#             message = quan
#             g = aggregate(message)
#             quanc = Quantized_topk(g + memc)
#             memc = g + memc - quanc
#             g = quanc
#             g = g.squeeze()
            
    
    
             #Quantize all the information, single pass + error feedback
#             quan = Quantized_l1sign(message + mem) 
#             mem = message + mem - quan
#             message = quan          
#             g = aggregate(message)


             #Quantize all the information, single pass
#             message1 = message[0:honestSize]
#             message2 = message[honestSize:]
#             quan1 = Quantized_randk(message1)
#             quan2 = Quantized_topk(message2)
#             message = torch.cat((quan1, quan2), 0)
#             g = aggregate(message)


            #DIANA type Quantization, single pass
            message1 = message[0:honestSize]
            message2 = message[honestSize:]
            delta1 = message1 - H1
            quan1 = Quantized_randk(delta1)
            message1 = H1 + quan1
            delta2 = message2 - H2
            quan2 = Quantized_topk(delta2)
            message2 = H2 + quan2
            H1 = H1 + alpha * quan1
            H2 = H2 + alpha * quan2
            message = torch.cat((message1, message2), 0)
            g = aggregate(message)
            
#             delta = message - H
#             quan = Quantized_randk(delta)
#             message = H + quan
#             H = H + alpha * quan
#             g = aggregate(message)
            
            #g = aggregate(message)
            w.add_(-gamma, g.data)
            
        loss = F(w, dataset, weight_decay)
        acc = accuracy(w, dataset)
        path.append(loss)
        var = getVarience(message, honestSize)
        variencePath.append(var)
        log('[SAGA]已迭代 {}/{} rounds (interval: {:.0f}), loss={:.9f}, accuracy={:.2f}, var={:.9f}'.format(
            r+1, rounds, displayInterval, loss, acc, var
        ))
    return w, path, variencePath

# Attacks

In [ ]:
def white(messages, byzantinesize):
    # 均值相同，方差较大
    mu = torch.mean(messages[0:-byzantinesize], dim=0)
    messages[-byzantinesize:].copy_(mu)
    noise = torch.randn((byzantinesize, messages.size(1)), dtype=torch.float64)
    messages[-byzantinesize:].add_(30, noise)
def maxValue(messages, byzantinesize):
    mu = torch.mean(messages[0:-byzantinesize], dim=0)
    meliciousMessage = -3*mu
    messages[-byzantinesize:].copy_(meliciousMessage)
def zeroGradient(messages, byzantinesize):
    s = torch.sum(messages[0:-byzantinesize], dim=0)
    messages[-byzantinesize:].copy_(-s / byzantinesize)

# main function

In [ ]:
import traceback
def run(optimizer, aggregate, attack, config, recordInFile=True, markOnTitle='cd_randk0.001'):
    
    if attack == None:
        title = '{}_{}_{}'.format(optimizer.__name__, 'baseline', aggregate.__name__)
    else:
        title = '{}_{}_{}'.format(optimizer.__name__, attack.__name__, aggregate.__name__)
    if markOnTitle != '':
        title = title + '_' + markOnTitle
    print(dataSetConfig['name'] + '_' + title)
    print('Fmin={}'.format(Fmin))

    _config = config.copy()
    _config['aggregate'] = aggregate
    _config['attack'] = attack
    attackName = 'baseline' if attack == None else attack.__name__
    if attack == None:
        _config['byzantineSize'] = 0
        
    # 打印运行信息
    print('[提交任务] ' + dataSetConfig['name'] + '_' + title)
    print('[运行信息]')
    print('{:7s} name={} aggregation={} attack={}'.format('[优化方法]', optimizer.__name__, aggregate.__name__, attackName))
    print('{:7s} gamma={} weight_decay={}'.format('[优化器设置]', _config['gamma'], _config['weight_decay']))
    print('{:7s} honestSize={}, byzantineSize={}'.format('[节点个数]', _config['honestSize'], _config['byzantineSize']))
    print('{:7s} rounds={}, displayInterval={}'.format('[运行次数]', _config['rounds'], _config['displayInterval']))
    print('{:7s} SEED={}, fixSeed={}'.format('[torch设置]', _config['SEED'], _config['fixSeed']))
    print('-------------------------------------------')
    
    log('提交任务')
    try:
        w, path, variancePath = optimizer(w0, **_config)

        record = {
            **dataSetConfig,
            'gamma': _config['gamma'],
            'weight_decay': _config['weight_decay'],
            'honestSize': _config['honestSize'],
            'byzantineSize': _config['byzantineSize'],
            'rounds': _config['rounds'],
            'displayInterval': _config['displayInterval'],
            'path': path,
            'variancePath': variancePath,
        }

        if recordInFile:
            with open(CACHE_DIR + title, 'wb') as f:
                pickle.dump(record, f)

        axis = plt.axes()
        plt.plot(list(range(len(path))), logAxis(path, Fmin))
        axis.set_yscale('log')
    except Exception as e:
        traceback.print_exc()

# 运行实验

## 正确性测试

出现函数到达最小值后，重新回弹的现象，原因可能有
1. 目标函数写错：忘记加惩罚项，忘记除以二等
2. 触及机器精度边界

## 计算最小值

In [ ]:
run(optimizer = SAGA, aggregate = mean, attack = None, config = SAGAConfig, recordInFile = False)

从零开始跑

In [ ]:
_VRConfig = VRConfig.copy()
_VRConfig['epoch'] = 10
_VRConfig['gamma'] = 2e-2
w_min = SAGA_min(w0, dataset, **_VRConfig)
Fmin = F(w_min, dataset, _VRConfig['weight_decay'])
print(Fmin)

精度不够继续跑

In [ ]:
_VRConfig = VRConfig.copy()
# _VRConfig['epoch'] = dataSetConfig['epoch'] * HONEST_SIZE
_VRConfig['epoch'] = 20
_VRConfig['gamma'] = 2e-2
w_min = SAGA_min(w_min, dataset, **_VRConfig)
Fmin = F(w_min, dataset, _VRConfig['weight_decay'])
print(Fmin)

存储Fmin

In [ ]:
#  with open(CACHE_DIR + 'Fmin', 'wb') as f:
#      pickle.dump({
#          'Fmin': Fmin,
#          'w_min': w_min
#      }, f)

读取Fmin

In [ ]:
with open(CACHE_DIR + 'Fmin', 'rb') as f:
    obj = pickle.load(f)
    Fmin, w_min = obj['Fmin'], obj['w_min']

## SGD

### SGD - mean

In [ ]:
run(optimizer = SGD, aggregate = mean, attack = None, config = SGDConfig)

white

In [ ]:
run(optimizer = SGD, aggregate = mean, attack = white, config = SGDConfig)

max

In [ ]:
run(optimizer = SGD, aggregate = mean, attack = maxValue, config = SGDConfig)

zero Gradient

In [ ]:
run(optimizer = SGD, aggregate = mean, attack = zeroGradient, config = SGDConfig)

### SGD - geomtric median

In [ ]:
run(optimizer = SGD, aggregate = gm, attack = None, config = SGDConfig)

white

In [ ]:
run(optimizer = SGD, aggregate = gm, attack = white, config = SGDConfig)

max

In [ ]:
run(optimizer = SGD, aggregate = gm, attack = maxValue, config = SGDConfig)

zero Gradient

In [ ]:
run(optimizer = SGD, aggregate = gm, attack = zeroGradient, config = SGDConfig)

### SGD - Krum

baseline

In [ ]:
Krum = Krum_(nodeSize=VRConfig['honestSize'], byzantineSize=0)
run(optimizer = SGD, aggregate = Krum, attack = None, config = SGDConfig)

white

In [ ]:
Krum = Krum_(nodeSize=VRConfig['honestSize'], byzantineSize=VRConfig['byzantineSize'])
run(optimizer = SGD, aggregate = Krum, attack = white, config = SGDConfig)

max

In [ ]:
Krum = Krum_(nodeSize=VRConfig['honestSize'], byzantineSize=VRConfig['byzantineSize'])
run(optimizer = SGD, aggregate = Krum, attack = maxValue, config = SGDConfig)

zero Gradient

In [ ]:
Krum = Krum_(nodeSize=VRConfig['honestSize'], byzantineSize=VRConfig['byzantineSize'])
run(optimizer = SGD, aggregate = Krum, attack = zeroGradient, config = SGDConfig)

### SGD - Median

In [ ]:
run(optimizer = SGD, aggregate = median, attack = None, config = SGDConfig)

white

In [ ]:
run(optimizer = SGD, aggregate = median, attack = white, config = SGDConfig)

max

In [ ]:
run(optimizer = SGD, aggregate = median, attack = maxValue, config = SGDConfig)

zero Gradient

In [ ]:
run(optimizer = SGD, aggregate = median, attack = zeroGradient, config = SGDConfig)

## BatchSGD

### BatchSGD - mean

In [ ]:
log('Fmin={}'.format(Fmin))

_VRConfig = batchConfig.copy()
_VRConfig['aggregate'] = aggregate_linear
_VRConfig['attack'] = None
_VRConfig['byzantineSize'] = 0

w, path, variancePath = FedBatchSGD(w0, **_VRConfig)

record = {
    **dataSetConfig,
    'gamma': _VRConfig['gamma'],
    'batchSize': _VRConfig['batchSize'],
    'path': path,
    'variancePath': variancePath,
}

with open(CACHE_DIR + 'BatchSGD_baseline_mean', 'wb') as f:
    pickle.dump(record, f)
    
axis = plt.axes()
plt.plot(list(range(len(path))), logAxis(path, Fmin))
axis.set_yscale('log')

white

In [ ]:
log('Fmin={}'.format(Fmin))

_VRConfig = batchConfig.copy()
_VRConfig['aggregate'] = aggregate_linear
_VRConfig['attack'] = whiteNoise
w, path, variancePath = FedBatchSGD(w0, **_VRConfig)

record = {
    **dataSetConfig,
    'gamma': _VRConfig['gamma'],
    'batchSize': _VRConfig['batchSize'],
    'path': path,
    'variancePath': variancePath,
}

with open(CACHE_DIR + 'BatchSGD_white_mean', 'wb') as f:
    pickle.dump(record, f)
    
axis = plt.axes()
plt.plot(list(range(len(path))), logAxis(path, Fmin))
axis.set_yscale('log')

max

In [ ]:
log('Fmin={}'.format(Fmin))

_VRConfig = batchConfig.copy()
_VRConfig['aggregate'] = aggregate_linear
_VRConfig['attack'] = maxValue
w, path, variancePath = FedBatchSGD(w0, **_VRConfig)

record = {
    **dataSetConfig,
    'gamma': _VRConfig['gamma'],
    'batchSize': _VRConfig['batchSize'],
    'path': path,
    'variancePath': variancePath,
}

with open(CACHE_DIR + 'BatchSGD_maxValue_mean', 'wb') as f:
    pickle.dump(record, f)
    
axis = plt.axes()
plt.plot(list(range(len(path))), logAxis(path, Fmin))
axis.set_yscale('log')

zero Gradient

In [ ]:
log('Fmin={}'.format(Fmin))

_VRConfig = batchConfig.copy()
_VRConfig['aggregate'] = aggregate_linear
_VRConfig['attack'] = zeroGradient
w, path, variancePath = FedBatchSGD(w0, **_VRConfig)

record = {
    **dataSetConfig,
    'gamma': _VRConfig['gamma'],
    'batchSize': _VRConfig['batchSize'],
    'path': path,
    'variancePath': variancePath,
}

with open(CACHE_DIR + 'BatchSGD_zeroGradient_mean', 'wb') as f:
    pickle.dump(record, f)
    
axis = plt.axes()
plt.plot(list(range(len(path))), logAxis(path, Fmin))
axis.set_yscale('log')

### BatchSGD - geomtric median

In [ ]:
log('Fmin={}'.format(Fmin))

_VRConfig = batchConfig.copy()
_VRConfig['aggregate'] = aggregate_geometric
_VRConfig['attack'] = None
_VRConfig['byzantineSize'] = 0
w, path, variancePath = FedBatchSGD(w0, **_VRConfig)

record = {
    **dataSetConfig,
    'gamma': _VRConfig['gamma'],
    'batchSize': _VRConfig['batchSize'],
    'path': path,
    'variancePath': variancePath,
}

with open(CACHE_DIR + 'BatchSGD_baseline_gm', 'wb') as f:
    pickle.dump(record, f)
    
axis = plt.axes()
plt.plot(list(range(len(path))), logAxis(path, Fmin))
axis.set_yscale('log')

white

In [ ]:
log('Fmin={}'.format(Fmin))

_VRConfig = batchConfig.copy()
_VRConfig['aggregate'] = aggregate_geometric
_VRConfig['attack'] = whiteNoise
w, path, variancePath = FedBatchSGD(w0, **_VRConfig)

record = {
    **dataSetConfig,
    'gamma': _VRConfig['gamma'],
    'batchSize': _VRConfig['batchSize'],
    'path': path,
    'variancePath': variancePath,
}

with open(CACHE_DIR + 'BatchSGD_white_gm', 'wb') as f:
    pickle.dump(record, f)
    
axis = plt.axes()
plt.plot(list(range(len(path))), logAxis(path, Fmin))
axis.set_yscale('log')

max

In [ ]:
log('Fmin={}'.format(Fmin))

_VRConfig = batchConfig.copy()
_VRConfig['aggregate'] = aggregate_geometric
_VRConfig['attack'] = maxValue
w, path, variancePath = FedBatchSGD(w0, **_VRConfig)

record = {
    **dataSetConfig,
    'gamma': _VRConfig['gamma'],
    'batchSize': _VRConfig['batchSize'],
    'path': path,
    'variancePath': variancePath,
}

with open(CACHE_DIR + 'BatchSGD_maxValue_gm', 'wb') as f:
    pickle.dump(record, f)
    
axis = plt.axes()
plt.plot(list(range(len(path))), logAxis(path, Fmin))
axis.set_yscale('log')

zero Gradient

In [ ]:
log('Fmin={}'.format(Fmin))

_VRConfig = batchConfig.copy()
_VRConfig['aggregate'] = aggregate_geometric
_VRConfig['attack'] = zeroGradient
w, path, variancePath = FedBatchSGD(w0, **_VRConfig)

record = {
    **dataSetConfig,
    'gamma': _VRConfig['gamma'],
    'batchSize': _VRConfig['batchSize'],
    'path': path,
    'variancePath': variancePath,
}

with open(CACHE_DIR + 'BatchSGD_zeroGradient_gm', 'wb') as f:
    pickle.dump(record, f)
    
axis = plt.axes()
plt.plot(list(range(len(path))), logAxis(path, Fmin))
axis.set_yscale('log')

## SAGA

### SAGA - mean

In [ ]:
run(optimizer = SAGA, aggregate = mean, attack = None, config = SAGAConfig)

white

In [ ]:
run(optimizer = SAGA, aggregate = mean, attack = white, config = SAGAConfig)

max

In [ ]:
run(optimizer = SAGA, aggregate = mean, attack = maxValue, config = SAGAConfig)

zero Gradient

In [ ]:
run(optimizer = SAGA, aggregate = mean, attack = zeroGradient, config = SAGAConfig)

### SAGA - geomtric median

baseline

In [ ]:
run(optimizer = SAGA, aggregate = gm, attack = None, config = SAGAConfig)

white

In [ ]:
run(optimizer = SAGA, aggregate = gm, attack = white, config = SAGAConfig)

max

In [ ]:
run(optimizer = SAGA, aggregate = gm, attack = maxValue, config = SAGAConfig)

zero Gradient

In [ ]:
run(optimizer = SAGA, aggregate = gm, attack = zeroGradient, config = SAGAConfig)